In [1]:
### Network construction
# data file: preprocessed_data.pkl
# bipartite network user - tweet
# one-mode user network 
# one-mode tweet network

# other combinations:
# one-mode pro-russian / pro-ukrainian user network   
# one-mode user network before / after Twitter ban in Russia   

# export nodelist and edgelist for network visualisation

In [46]:
import pandas as pd
import networkx as nx
import numpy as np
import csv
from networkx.algorithms import bipartite

In [3]:
data = pd.read_pickle("preprocessed_data.pkl")
data.head()

userid       username  textid         location language  \
0  1497277564200517635  GulambiLamver       0     गुलाबी_स्तान       en   
1           4832941924    Mari_Berbec       1  Berlin, Germany       en   
2           2314945207  Outspoken_Sam       2       Texas, USA       en   
3           2811043316      joncreed2       3  Nottinghamshire      und   
4  1239053406301335552    PChauhan_96       4    Chamba, India       en   

               tweetid       tweetcreatedts  retweetcount  \
0  1497433817086234624  2022-02-26 04:50:17             0   
1  1497476520742309891  2022-02-26 07:39:58             0   
2  1497066569263767555  2022-02-25 04:30:58             0   
3  1497308839401635845  2022-02-25 20:33:40            16   
4  1497551063913619457  2022-02-26 12:36:11           276   

                                            hashtags  \
0  [{'text': 'Ukraine', 'indices': [40, 48]}, {'t...   
1  [{'text': 'putin', 'indices': [53, 59]}, {'tex...   
2       [{'text': 'Ukraine', 'indices': [272, 280]}]   
3  [{'text': 'Ukraine', 'indices': [20, 28]}, {'t...   
4         [{'text': 'Ukraine', 'indices': [34, 42]}]   

                                                text       usercreatedts  \
0  Coz barking is our fundamental right..\n\n#Ukr... 2022-02-25 18:29:39   
1  @charlieweissr Socialism has nothing to do wit... 2016-01-21 13:57:12   
2  We're seeing repeated demonstrations of who Bi... 2014-01-28 05:46:59   
3  #Ukraine #Nottingham #nottinghamtogether #Stan... 2014-09-15 10:10:32   
4  That's a Leader ❣️ \n#Ukraine https://t.co/BpF... 2020-03-15 04:59:11   

   following  followers  totaltweets  \
0          4          0            2   
1        921        224         2848   
2       1162        642         3869   
3        778        361         3960   
4         25          6          374   

                                       tags_splitted  \
0  [Ukraine, UkraineUnderAttack, RussiaUkraineWar...   
1                                     [putin, trump]   
2                                          [Ukraine]   
3  [Ukraine, Nottingham, nottinghamtogether, Stan...   
4                                          [Ukraine]   

                                          tags_lower  pro_Russian  \
0  [ukraine, ukraineunderattack, russiaukrainewar...        False   
1                                     [putin, trump]        False   
2                                          [ukraine]        False   
3  [ukraine, nottingham, nottinghamtogether, stan...        False   
4                                          [ukraine]        False   

   pro_Ukraine  Neutral     NodeType  
0         True     True  pro_Ukraine  
1        False     True      Neutral  
2        False     True      Neutral  
3         True     True  pro_Ukraine  
4        False     True      Neutral

In [4]:
# Get nodes for bipartite network
users = data["userid"]
tweets = data["textid"]

# Create edge list for bipartite network
columns = ["userid", "textid"]
edges = [(user, textid) for user, textid in data[columns].values]

# save edge list to file
# with open('edges.csv','wb') as file:
#     csv_out=csv.writer(out)
#     csv_out.writerow(['name','num'])
#     for row in data:
#         csv_out.writerow(row)

In [5]:
print(users.nunique())
print(tweets.nunique())

130608
105158


In [6]:
# bipartite network construction
B = nx.Graph()
B.name = "Bipartite user-tweet network"
B.add_nodes_from(users, bipartite=0)
B.add_nodes_from(tweets, bipartite=1)
B.add_edges_from(edges)

In [7]:
nx.info(B)

"Graph named 'Bipartite user-tweet network' with 235745 nodes and 161429 edges"

In [8]:
users_nodes = {n for n, a in B.nodes(data=True) if a["bipartite"] == 0}
tweets_nodes = set(B) - users_nodes

B_users = bipartite.projected_graph(B, users_nodes)
B_users.name = "One-mode user network"
B_tweets = bipartite.projected_graph(B, tweets_nodes)
B_tweets.name = "One-mode tweet network"

print(nx.info(B_users))
print(nx.info(B_tweets))

Graph named 'One-mode user network' with 130619 nodes and 1436926 edges
Graph named 'One-mode tweet network' with 106650 nodes and 78517 edges


## Compare two timeframes

In [9]:
# Filter tweets by time period: before and after the tweeter ban 
time_before = data[(data['tweetcreatedts'] < '2022-03-04 23:59:59+00:00')]
time_after = data[(data['tweetcreatedts'] > '2022-03-04 23:59:59+00:00')]
print("rows of timeframe 1 dataset:",len(time_before))
print("rows of timeframe 2 dataset:",len(time_after))

rows of timeframe 1 dataset: 73333
rows of timeframe 2 dataset: 88295


### (1) timeframe1: before

In [10]:
# Get nodes for bipartite network
users_1 = time_before["userid"]
tweets_1 = time_before["textid"]

# Create edge list for bipartite network
columns = ["userid", "textid"]
edges_1 = time_before[columns]
edges_1.to_csv("edgelist_t1.csv",sep = ",", header = True, encoding = "UTF-8",index=False)
# same list-type edgelist for network construction
edges_1 = [(user, textid) for user, textid in time_before[columns].values]

print(users_1.nunique(),"unique users in timeframe 1")
print(tweets_1.nunique(),"unique tweets in timeframe 1")

66530 unique users in timeframe 1
49702 unique tweets in timeframe 1


In [11]:
# construct projected one-mode network for users
# export edgelist for visualisation
B1 = nx.Graph()
B1.name = "Bipartite user-tweet network time 1"
B1.add_nodes_from(users_1, bipartite=0)
B1.add_nodes_from(tweets_1, bipartite=1)
B1.add_edges_from(edges_1)

users_nodes = {n for n, a in B1.nodes(data=True) if a["bipartite"] == 0}
tweets_nodes = set(B1) - users_nodes

B1_users = bipartite.weighted_projected_graph(B1, users_nodes)
B1_users.name = "One-mode user network 1"
B1_tweets = bipartite.weighted_projected_graph(B1, tweets_nodes)
B1_tweets.name = "One-mode tweet network 1"

print(nx.info(B1_users))
print(nx.info(B1_tweets))

Graph named 'One-mode user network 1' with 66532 nodes and 403244 edges
Graph named 'One-mode tweet network 1' with 49760 nodes and 10645 edges


In [ ]:
# write edgelist of projected network
nx.write_weighted_edgelist(B1_users, "projected_w_user_edgelist_1.csv", delimiter=',', encoding='utf-8')   
# format the edgelist file for Gephi
gephi1_edgelist = pd.read_csv("projected_w_user_edgelist_1.csv",header=None)
gephi1_edgelist.columns=['Source','Target','weight']
gephi1_edgelist.to_csv("projected_w_user_edgelist_1.csv",sep = ",", header = True, encoding = "UTF-8",index=False)


### Eigenvector centrality
https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.eigenvector_centrality.html

In [64]:
eigen_c1 = nx.eigenvector_centrality(B1_users, max_iter=100)
#eigen_c1 = dict(sorted(eigen_c1.items(),key= lambda x:x[1]))
len(eigen_c1)
# covert dic to dataframe 
eigen_c1 = pd.DataFrame(list(eigen_c1.items()),columns = ['userid','Eigen_c']) 
eigen_c1

userid       Eigen_c
0      1255073685158887424  2.567680e-19
1       956102521503744001  2.006000e-21
2      1285434543344844802  2.006000e-21
3      1287813629845307395  1.233965e-09
4       783003137183449092  1.266071e-11
...                    ...           ...
66527                29283  3.767417e-07
66528                32802  2.968077e-17
66529                38153  2.567680e-19
66530                41298  2.567680e-19
66531                12418  2.567680e-19

[66532 rows x 2 columns]

### (2) timeframe2: After

In [27]:
# Get nodes for bipartite network
users_2 = time_after["userid"]
tweets_2 = time_after["textid"]

# Create edge list for bipartite network
columns = ["userid", "textid"]
edges_2 = time_after[columns]
edges_2.to_csv("edgelist_t2.csv",sep = ",", header = True, encoding = "UTF-8",index=False)
# same list-type edgelist for network construction
edges_2 = [(user, textid) for user, textid in time_after[columns].values]

print(users_2.nunique(),"unique users in timeframe 2")
print(tweets_2.nunique(),"unique tweets in timeframe 2")

72022 unique users in timeframe 2
56463 unique tweets in timeframe 2


In [28]:
# construct projected one-mode network for users
# export edgelist for visualisation
B2 = nx.Graph()
B2.name = "Bipartite user-tweet network time 2"
B2.add_nodes_from(users_2, bipartite=0)
B2.add_nodes_from(tweets_2, bipartite=1)
B2.add_edges_from(edges_2)

users_nodes = {n for n, a in B2.nodes(data=True) if a["bipartite"] == 0}
tweets_nodes = set(B2) - users_nodes

B2_users = bipartite.weighted_projected_graph(B2, users_nodes)
B2_users.name = "One-mode user network 2"
B2_tweets = bipartite.weighted_projected_graph(B2, tweets_nodes)
B2_tweets.name = "One-mode tweet network 2"

print(nx.info(B2_users))
print(nx.info(B2_tweets))

Graph named 'One-mode user network 2' with 72025 nodes and 959639 edges
Graph named 'One-mode tweet network 2' with 57184 nodes and 40248 edges


In [ ]:
# write edgelist of projected network
nx.write_weighted_edgelist(B2_users, "projected_w_user_edgelist_2.csv", delimiter=',', encoding='utf-8') 
# format the edgelist file for Gephi
gephi2_edgelist = pd.read_csv("projected_w_user_edgelist_2.csv",header=None)
gephi2_edgelist.columns=['Source','Target','weight']
gephi2_edgelist.to_csv("projected_w_user_edgelist_2.csv",sep = ",", header = True, encoding = "UTF-8",index=False)


### Eigenvector centrality

In [65]:
eigen_c2 = nx.eigenvector_centrality(B2_users, max_iter=100)
#engen_c2 = dict(sorted(eigen_c2.items(),key= lambda x:x[1]))
len(engen_c2)
# covert dic to dataframe 
eigen_c2 = pd.DataFrame(list(eigen_c2.items()),columns = ['userid','Eigen_c']) 
#eigen_c2

## Export nodelist

### Mark the nodes appeared in the projected user network

In [52]:
nodelist1 = list(B1_users.nodes)
nodelist2 = list(B2_users.nodes)

# lambda functions marking political orientation if related hashtags were found 
node1_check = lambda node: int(node) in nodelist1 # return true if userid of the nodes in overlap list
node2_check = lambda node: int(node) in nodelist2

data['nodeInG1'] = data['userid'].apply(node1_check)
data['nodeInG2'] = data['userid'].apply(node2_check)

# format node ID for Gephi
data["ID"] = data["userid"].apply(lambda x: int(x))

In [54]:
data.head(3)

userid       username  textid         location language  \
0  1497277564200517635  GulambiLamver       0     गुलाबी_स्तान       en   
1           4832941924    Mari_Berbec       1  Berlin, Germany       en   
2           2314945207  Outspoken_Sam       2       Texas, USA       en   

               tweetid       tweetcreatedts  retweetcount  \
0  1497433817086234624  2022-02-26 04:50:17             0   
1  1497476520742309891  2022-02-26 07:39:58             0   
2  1497066569263767555  2022-02-25 04:30:58             0   

                                            hashtags  \
0  [{'text': 'Ukraine', 'indices': [40, 48]}, {'t...   
1  [{'text': 'putin', 'indices': [53, 59]}, {'tex...   
2       [{'text': 'Ukraine', 'indices': [272, 280]}]   

                                                text  ... pro_Ukraine  \
0  Coz barking is our fundamental right..\n\n#Ukr...  ...        True   
1  @charlieweissr Socialism has nothing to do wit...  ...       False   
2  We're seeing repeated demonstrations of who Bi...  ...       False   

   Neutral     NodeType  R_i U_i N_i  polar_index  nodeInG1  nodeInG2  \
0     True  pro_Ukraine    0   1   1            1      True     False   
1     True      Neutral    0   0   1            0      True      True   
2     True      Neutral    0   0   1            0      True     False   

                    ID  
0  1497277564200517635  
1           4832941924  
2           2314945207  

[3 rows x 27 columns]

### Construct a political orientation index for ID

In [57]:
data['R_i'] = np.where(data['pro_Russian'],1,0)
data['U_i'] = np.where(data['pro_Ukraine'],1,0)
data['N_i'] = np.where(data['Neutral'],1,0)

#data.groupby('userid',as_index=False)['R_i'].sum().sort_values(by=['R_i'],ascending=False)
#data.groupby('userid',as_index=False)['U_i'].sum().sort_values(by=['U_i'],ascending=False)
#data.groupby('userid',as_index=False)['N_i'].sum().sort_values(by=['N_i'],ascending=False)
data["polar_i"] = data["U_i"] - data["R_i"]
polar_index = data.groupby('userid',as_index=False)['polar_i'].sum().sort_values(by=['polar_i'],ascending=False)
# positive score: pro-Ukraine, negative score: pro-Russian
polar_index

userid  polar_i
11706              88196314       54
60789            4503483681       47
89284   1203552378382934016       44
88935   1197961849314168833       41
129023  1499763123603050497       39
...                     ...      ...
49258            2435380381       -2
98928   1307865457903448064       -3
8582               53665207       -3
120974  1484008649916051457       -3
129372  1499965514227060740       -5

[130608 rows x 2 columns]

### Add Eigenvector and political orientation index to nodelist for Gephi

In [66]:
# select info from pre-processed and labelled dataframe
node_columns = ["userid","username","usercreatedts","following","followers","totaltweets","NodeType","ID"]

nodelist1 = data[data['nodeInG1']][node_columns]
nodelist2 = data[data['nodeInG2']][node_columns]

nodelist1 = pd.merge(nodelist1, polar_index, how = 'left',on='userid')
nodelist2 = pd.merge(nodelist2, polar_index, how = 'left',on='userid')

nodelist1 = pd.merge(nodelist1, eigen_c1, how='left',on="userid")
nodelist2 = pd.merge(nodelist2, eigen_c2, how='left',on="userid")

print(nodelist1["ID"].nunique(),"unique users identified in gephi graph in timeframe 1")
print(nodelist2["ID"].nunique(),"unique users identified in gephi graph in timeframe 2")

66527 unique users identified in gephi graph in timeframe 1
72019 unique users identified in gephi graph in timeframe 2


In [67]:
nodelist1.to_csv("nodelist1.csv",sep = ",", header = True, encoding = "UTF-8",index=False)
nodelist2.to_csv("nodelist2.csv",sep = ",", header = True, encoding = "UTF-8",index=False)